In [61]:
%matplotlib inline

from __future__ import division

import dfmaker
import modelmaker
import AnaFunc

reload(dfmaker)
reload(modelmaker)
reload(AnaFunc)

import time

from sklearn import cross_validation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC

import numpy as np
import re
import pickle

import matplotlib.pyplot as plt
import pandas as pd

from sklearn import linear_model
from sklearn import metrics

In [62]:
def MakeNorms(df,v1,v2):
    
    x1,x2,y = modelmaker.retrieve_vocab(v1,v2,df)

    norm1 = np.zeros(x1.shape[0])
    for i,x in enumerate(x1):
        norm1[i] = np.linalg.norm(x)

    norm2 = np.zeros(x2.shape[0])
    for i,x in enumerate(x2):
        norm2[i] = np.linalg.norm(x)

    return norm1,norm2

In [79]:
def TestLog(df_test,v1,v2):

    #print df_test
    
    norm1,norm2 = MakeNorms(df_test,v1,v2)

    results = ((norm1 > 5) | (norm2 > 0))
    
    #print results.sum()
    
    return results
    

In [80]:
def snipstr(x,mystr):
    x = re.sub("\S*%s\S*" % mystr,"",x)
    return x

def remove_word(df,mylist):
    for w in mylist:
        df['taglist'] = df['taglist'].apply(lambda x, y: snipstr(x,y), args=(w,))
        df['words'] = df['words'].apply(lambda x, y: snipstr(x,y), args=(w,))
    return df

In [81]:
def fullpipe(tag,nolist,myconfig,vect_text,vect_tag,unmarked=False,wlist=[]):

    df_class1 = dfmaker.get_train_dfs(tag,myconfig)
    df_class1['evtclass'] = 1
 
    _,_,_,v1,v2 = modelmaker.train_vocab(vect_text,vect_tag, \
                                         df_class1,downsample=False)

    for notag in nolist:
        
        df_class0 = dfmaker.get_train_dfs(notag,myconfig)
        df_class0['evtclass'] = 0

        df_test = pd.concat([df_class0,df_class1])

        if unmarked:
            df_test = remove_word(df_test,wlist)
        
        y_real = df_test['evtclass'].values
        y_pred = TestLog(df_test,v1,v2)
        
        tot_pos = (y_real==1).sum()
        tot_neg = (y_real==0).sum()
    
        tpos = ((y_pred == 1) * (y_real == 1 )).sum()
        tpr = tpos/tot_pos
        fpos = ((y_pred == 1) * (y_real == 0 )).sum()
        fpr = fpos/tot_neg
        
        print "true=%s, false=%s, TPR=%f, FPR=%f" % (tag,notag,tpr,fpr)
        
        path = "/Users/ruthtoner/CodingMacros/ProjectInsight/Fanguard/files"
        v1name = "pfilter_%s_v1.pkl" % tag
        v2name = "pfilter_%s_v2.pkl" % tag 

        with open('%s/%s' % (path,v1name), 'wb') as fid:
            pickle.dump(v1, fid)   
        with open('%s/%s' % (path,v2name), 'wb') as fid:
            pickle.dump(v2, fid)  
        

In [82]:
myconfig = "/Users/ruthtoner/CodingMacros/ProjectInsight/myconfigs.cfg"

#Vectorizer:
vect_text = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 50, \
                             max_df = 1.0)

vect_tag = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 50, \
                             max_df = 1.0, \
                             binary = True)

model_logreg = linear_model.LogisticRegression(C=1)

In [83]:
nolist = ['dai','utale','lis','slock','got','han','aou','madmax']
print "Marked:"
fullpipe("sw",nolist,myconfig,vect_text,vect_tag)
print "Unmarked:"
fullpipe("sw",nolist,myconfig,vect_text,vect_tag,True,['star','war'])

Marked:
true=sw, false=dai, TPR=1.000000, FPR=0.269592
true=sw, false=utale, TPR=1.000000, FPR=0.373466
true=sw, false=lis, TPR=1.000000, FPR=0.359100
true=sw, false=slock, TPR=1.000000, FPR=0.265769
true=sw, false=got, TPR=1.000000, FPR=0.188856
true=sw, false=han, TPR=1.000000, FPR=0.200056
true=sw, false=aou, TPR=1.000000, FPR=0.322480
true=sw, false=madmax, TPR=1.000000, FPR=0.410067
Unmarked:
true=sw, false=dai, TPR=0.755232, FPR=0.268337
true=sw, false=utale, TPR=0.755232, FPR=0.371998
true=sw, false=lis, TPR=0.755232, FPR=0.358567
true=sw, false=slock, TPR=0.755232, FPR=0.264349
true=sw, false=got, TPR=0.755232, FPR=0.186294
true=sw, false=han, TPR=0.755232, FPR=0.199194
true=sw, false=aou, TPR=0.755232, FPR=0.312719
true=sw, false=madmax, TPR=0.755232, FPR=0.384564


In [84]:
nolist = ['sw','utale','lis','slock','got','han','aou','madmax']
print "Marked:"
fullpipe("dai",nolist,myconfig,vect_text,vect_tag)
print "Unmarked:"
fullpipe("dai",nolist,myconfig,vect_text,vect_tag,True,['dai'])

Marked:
true=dai, false=sw, TPR=1.000000, FPR=0.316140
true=dai, false=utale, TPR=1.000000, FPR=0.448397
true=dai, false=lis, TPR=1.000000, FPR=0.357888
true=dai, false=slock, TPR=1.000000, FPR=0.208067
true=dai, false=got, TPR=1.000000, FPR=1.000000
true=dai, false=han, TPR=1.000000, FPR=0.225970
true=dai, false=aou, TPR=1.000000, FPR=1.000000
true=dai, false=madmax, TPR=1.000000, FPR=0.351678
Unmarked:
true=dai, false=sw, TPR=0.861169, FPR=0.316116
true=dai, false=utale, TPR=0.861169, FPR=0.448397
true=dai, false=lis, TPR=0.861169, FPR=0.357888
true=dai, false=slock, TPR=0.861169, FPR=0.208067
true=dai, false=got, TPR=0.861169, FPR=1.000000
true=dai, false=han, TPR=0.861169, FPR=0.225970
true=dai, false=aou, TPR=0.861169, FPR=1.000000
true=dai, false=madmax, TPR=0.861169, FPR=0.351007


In [85]:
nolist = ['sw','dai','lis','slock','got','han','aou','madmax']
print "Marked:"
fullpipe("utale",nolist,myconfig,vect_text,vect_tag)
print "Unmarked:"
fullpipe("utale",nolist,myconfig,vect_text,vect_tag,True,['utale'])

Marked:
true=utale, false=sw, TPR=1.000000, FPR=0.307065
true=utale, false=dai, TPR=1.000000, FPR=0.368782
true=utale, false=lis, TPR=1.000000, FPR=0.354444
true=utale, false=slock, TPR=1.000000, FPR=0.186822
true=utale, false=got, TPR=1.000000, FPR=1.000000
true=utale, false=han, TPR=1.000000, FPR=0.191158
true=utale, false=aou, TPR=1.000000, FPR=0.294580
true=utale, false=madmax, TPR=1.000000, FPR=0.342282
Unmarked:
true=utale, false=sw, TPR=1.000000, FPR=0.307065
true=utale, false=dai, TPR=1.000000, FPR=0.368782
true=utale, false=lis, TPR=1.000000, FPR=0.354444
true=utale, false=slock, TPR=1.000000, FPR=0.186822
true=utale, false=got, TPR=1.000000, FPR=1.000000
true=utale, false=han, TPR=1.000000, FPR=0.191158
true=utale, false=aou, TPR=1.000000, FPR=0.294580
true=utale, false=madmax, TPR=1.000000, FPR=0.342282


In [86]:
nolist = ['sw','dai','utale','slock','got','han','aou','madmax']
print "Marked:"
fullpipe("lis",nolist,myconfig,vect_text,vect_tag)
print "Unmarked:"
fullpipe("lis",nolist,myconfig,vect_text,vect_tag,True,['life','is','strange'])

Marked:
true=lis, false=sw, TPR=1.000000, FPR=0.340977
true=lis, false=dai, TPR=1.000000, FPR=0.366204
true=lis, false=utale, TPR=1.000000, FPR=0.431398
true=lis, false=slock, TPR=1.000000, FPR=0.228019
true=lis, false=got, TPR=1.000000, FPR=1.000000
true=lis, false=han, TPR=1.000000, FPR=0.293369
true=lis, false=aou, TPR=1.000000, FPR=0.298295
true=lis, false=madmax, TPR=1.000000, FPR=1.000000
Unmarked:
true=lis, false=sw, TPR=0.753601, FPR=0.311916
true=lis, false=dai, TPR=0.753601, FPR=0.361389
true=lis, false=utale, TPR=0.753601, FPR=0.427888
true=lis, false=slock, TPR=0.753601, FPR=0.211059
true=lis, false=got, TPR=0.753601, FPR=1.000000
true=lis, false=han, TPR=0.753601, FPR=0.283081
true=lis, false=aou, TPR=0.753601, FPR=0.291740
true=lis, false=madmax, TPR=0.753601, FPR=1.000000


In [87]:
nolist = ['sw','dai','utale','lis','got','han','aou','madmax']
print "Marked:"
fullpipe("slock",nolist,myconfig,vect_text,vect_tag)
print "Unmarked:"
fullpipe("slock",nolist,myconfig,vect_text,vect_tag,True,['sherlock'])

Marked:
true=slock, false=sw, TPR=1.000000, FPR=0.281963
true=slock, false=dai, TPR=1.000000, FPR=0.219913
true=slock, false=utale, TPR=1.000000, FPR=0.198392
true=slock, false=lis, TPR=1.000000, FPR=1.000000
true=slock, false=got, TPR=1.000000, FPR=0.250928
true=slock, false=han, TPR=1.000000, FPR=0.278660
true=slock, false=aou, TPR=1.000000, FPR=0.267410
true=slock, false=madmax, TPR=1.000000, FPR=0.257047
Unmarked:
true=slock, false=sw, TPR=0.701171, FPR=0.281697
true=slock, false=dai, TPR=0.701171, FPR=0.219845
true=slock, false=utale, TPR=0.701171, FPR=0.198392
true=slock, false=lis, TPR=0.701171, FPR=1.000000
true=slock, false=got, TPR=0.701171, FPR=0.250562
true=slock, false=han, TPR=0.701171, FPR=0.276992
true=slock, false=aou, TPR=0.701171, FPR=0.267337
true=slock, false=madmax, TPR=0.701171, FPR=0.256376


In [88]:
nolist = ['sw','dai','utale','lis','slock','han','aou','madmax']
print "Marked:"
fullpipe("got",nolist,myconfig,vect_text,vect_tag)
print "Unmarked:"
fullpipe("got",nolist,myconfig,vect_text,vect_tag,True,['game-of-thrones'])

Marked:
true=got, false=sw, TPR=1.000000, FPR=0.255847
true=got, false=dai, TPR=1.000000, FPR=0.697989
true=got, false=utale, TPR=1.000000, FPR=0.198987
true=got, false=lis, TPR=1.000000, FPR=0.361816
true=got, false=slock, TPR=1.000000, FPR=0.267265
true=got, false=han, TPR=1.000000, FPR=0.287112
true=got, false=aou, TPR=1.000000, FPR=0.270396
true=got, false=madmax, TPR=1.000000, FPR=0.265772
Unmarked:
true=got, false=sw, TPR=1.000000, FPR=0.255847
true=got, false=dai, TPR=1.000000, FPR=0.697989
true=got, false=utale, TPR=1.000000, FPR=0.198987
true=got, false=lis, TPR=1.000000, FPR=0.361816
true=got, false=slock, TPR=1.000000, FPR=0.267265
true=got, false=han, TPR=1.000000, FPR=0.287112
true=got, false=aou, TPR=1.000000, FPR=0.270396
true=got, false=madmax, TPR=1.000000, FPR=0.265772


In [89]:
nolist = ['sw','dai','utale','lis','slock','got','aou','madmax']
print "Marked:"
fullpipe("han",nolist,myconfig,vect_text,vect_tag)
print "Unmarked:"
fullpipe("han",nolist,myconfig,vect_text,vect_tag,True,['hannibal'])

Marked:
true=han, false=sw, TPR=1.000000, FPR=0.274239
true=han, false=dai, TPR=1.000000, FPR=0.697921
true=han, false=utale, TPR=1.000000, FPR=0.354999
true=han, false=lis, TPR=1.000000, FPR=0.280491
true=han, false=slock, TPR=1.000000, FPR=0.254614
true=han, false=got, TPR=1.000000, FPR=0.293782
true=han, false=aou, TPR=1.000000, FPR=0.258596
true=han, false=madmax, TPR=1.000000, FPR=0.305369
Unmarked:
true=han, false=sw, TPR=0.609426, FPR=0.274191
true=han, false=dai, TPR=0.609426, FPR=0.697921
true=han, false=utale, TPR=0.609426, FPR=0.354999
true=han, false=lis, TPR=0.609426, FPR=0.280394
true=han, false=slock, TPR=0.609426, FPR=0.252839
true=han, false=got, TPR=0.609426, FPR=0.293103
true=han, false=aou, TPR=0.609426, FPR=0.258596
true=han, false=madmax, TPR=0.609426, FPR=0.304698


In [90]:
nolist = ['sw','dai','utale','lis','slock','got','han','madmax']
print "Marked:"
fullpipe("aou",nolist,myconfig,vect_text,vect_tag)
print "Unmarked:"
fullpipe("aou",nolist,myconfig,vect_text,vect_tag,True,['age','of','ultron'])

Marked:
true=aou, false=sw, TPR=1.000000, FPR=0.292004
true=aou, false=dai, TPR=1.000000, FPR=0.695988
true=aou, false=utale, TPR=1.000000, FPR=0.354126
true=aou, false=lis, TPR=1.000000, FPR=0.238689
true=aou, false=slock, TPR=1.000000, FPR=0.155182
true=aou, false=got, TPR=1.000000, FPR=0.231188
true=aou, false=han, TPR=1.000000, FPR=0.159210
true=aou, false=madmax, TPR=1.000000, FPR=0.364430
Unmarked:
true=aou, false=sw, TPR=0.891317, FPR=0.291159
true=aou, false=dai, TPR=0.891317, FPR=0.246329
true=aou, false=utale, TPR=0.891317, FPR=0.353765
true=aou, false=lis, TPR=0.891317, FPR=0.237816
true=aou, false=slock, TPR=0.891317, FPR=0.155081
true=aou, false=got, TPR=0.891317, FPR=0.230665
true=aou, false=han, TPR=0.891317, FPR=0.158988
true=aou, false=madmax, TPR=0.891317, FPR=0.362416
